># For data-Cleaning and error reduction

In [ ]:
from tqdm.notebook import tqdm_notebook
from fuzzywuzzy import fuzz
import json
import os

In [ ]:
# directory path containing wikipedia extracted data 
# NOTE This directory must only contain wikipedia extracted data (using wikipedia_data.ipynb)
location_of_wiki_jsons = '../Data-Collector/data/Wikipedia/'

# directory path to save json file
saving_location = './'

In [ ]:
# get all files contained in location directory
files = os.listdir(location_of_wiki_jsons)

In [ ]:
# stop words list
list_remove = ['private','limited','limit','ed','limite','d','llp','company','pvt','ltd','lim','limi','ted','(opc)','(india)']
def return_best(dict1,dict2,cin):
    """Score the company dict on score calculated

    Args:
        dict1 (dictionary): Company1 with same wiki page
        dict2 ([type]): Company2 with same wikipage
        cin ([type]): CIN of the company

    Returns:
        dictionary: Of best matching company
    """
    for i in dict1:
        temp_dict = dict1[i]
    names = [temp_dict['name'].lower(), dict2['name'].lower(),dict2['wikipedia page'].lower()]
    filter_name = []
    for i in range(len(names)):
        # format for score calculation and remove stop words
        brokn = names[i].split(' ')
        for j in range(len(brokn)):
            if(brokn[j]in list_remove):
                brokn[j]=''
        brokn = list(filter(lambda x:len(x)!=0,brokn))
        filter_name.append(''.join(brokn))
    # calculate score of both companies with wikipedea page title
    score1 = fuzz.ratio(filter_name[0],filter_name[2])
    score2 = fuzz.ratio(filter_name[1],filter_name[2])
    if(score2>score1): return {cin:dict2}
    else: return dict1

In [ ]:
final_dict = {}
other_dict = {}
with tqdm_notebook(total=len(files)) as pbar:
    for i in files:
        with open(location_of_wiki_jsons+i) as f:
            data = json.load(f)
        for i in data:
            print(i)
            if(data[i]['wikipedia page url'] not in other_dict):
                other_dict[data[i]['wikipedia page url']]={i:data[i]}
            else:
                dict1 = other_dict[data[i]['wikipedia page url']]
                dict2 = data[i]
                other_dict[data[i]['wikipedia page url']] = return_best(dict1,dict2,i)
        pbar.update()

# update the final_dict with correct data
for j in other_dict:
    for i in other_dict[j]:
        final_dict[i]=other_dict[j][i]

# Dump data to data.json in saving directory
with open(saving_location+'data.json','w+') as f:
    json.dump(final_dict,f,indent=4)